In [3]:
import pandas as pd
import numpy as np

In [4]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ', 'Croatia': 'HR',  'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'UK'}

## Making the first,second,third and fourth columns of the dataframe as date,month,year and time

In [5]:
data=pd.DataFrame()
temp=pd.read_csv('../Data Sources/ENTSO-E/Load/Croatia.csv')
data['Day']=temp['Time (CET)'].str[:2]
data['Month']=temp['Time (CET)'].str[3:5]
data['Year']=temp['Time (CET)'].str[6:10]
data['Time']=temp['Time (CET)'].str[11:16]+' - '+temp['Time (CET)'].str[29:35]

### In the country speciic Load data set, Austria,Belgium,Germany,Hungary,Netherlands report data every 15 minutes. Therefore, these countries have 35044 data points per year. UK and Ireland report data every 30 minutes henece these countries have 17522 datapoints per year. All the others report every 1 hour hence have 8761 datapoints per year. In Genearion dataset, situation is same as abobe except Belgium reports hourly data hence have 8761 datapoints. In Transmission dataset, all countries report data hourly. Therefroe,it is easy if all the data are converted to hourly data. To do that in the countries with 35044 datapoints, mean is calculated in every successive 4 datapoints. In the countries with 17522 datapoints, mean is calculated in every successive 2 datapoints.`

In [6]:
def mean_calculator(df):
    length=len(df.index)
    if length==35044:
        divider=4
    elif length==17522:
        divider=2
    else:
        divider=1

    df=df.groupby(np.arange(length)//divider).mean()
    return(df)    

# Preparing Load Data

In [7]:
def load(countries):

    load_data=pd.DataFrame()
    load_data[['Day','Month','Year','Time']]=data[['Day','Month','Year','Time']]

    for country,abbr in countries.items():
        temp=pd.read_csv(f'../Data Sources/ENTSO-E/Load/{country}.csv')
        temp=mean_calculator(temp)
        load_data[f'{abbr}']=temp[f'Actual Total Load [MW] - {country} ({abbr})']
        
    return(load_data)

# Preparing Generation Data

In [8]:
def generation(countries):
    
    generation_data=pd.DataFrame()
    generation_data[['Day','Month','Year','Time']]=data[['Day','Month','Year','Time']]

    for country,abbr in countries.items():
        temp=pd.read_csv(f'../Data Sources/ENTSO-E/Generation/{country}.csv',low_memory=False)
        temp=mean_calculator(temp)

        fuels=np.array(temp.columns)
        for fuel in fuels:
            edited_fuel=fuel[:-26]
            generation_data[f'{abbr} - {edited_fuel}']=temp[fuel]
        generation_data[f'{abbr} - Total'] = generation_data.filter(like=abbr).sum(axis=1)

    return(generation_data)

In [9]:
generation(countries)

,Day,Month,Year,Time,AT - Biomass,AT - Fossil Gas,AT - Fossil Hard coal,AT - Fossil Oil,AT - Geothermal,AT - Hydro Pumped Storage,...,UK - Fossil Oil,UK - Hydro Pumped Storage,UK - Hydro Pumped Storage,UK - Hydro Run-of-river and poundage,UK - Nuclear,UK - Other,UK - Solar,UK - Wind Offshore,UK - Wind Onshore,UK - Total
0,01,01,2019,00:00 - 01:00,304.0,1269.75,155.50,0.0,0.0,0.00,...,3.0,0.0,NaN,428.5,6941.5,66.0,0.0,3131.0,6527.5,25144.5
1,01,01,2019,01:00 - 02:00,304.0,935.25,154.75,0.0,0.0,0.00,...,0.0,52.5,NaN,404.0,6899.0,64.0,0.0,3198.0,6201.5,23967.5
2,01,01,2019,02:00 - 03:00,304.0,611.00,155.25,0.0,0.0,0.00,...,0.0,0.0,NaN,373.5,6830.5,63.0,0.0,3277.5,6095.0,23512.0
3,01,01,2019,03:00 - 04:00,304.0,568.00,155.25,0.0,0.0,0.00,...,0.0,0.0,NaN,349.0,6826.0,63.0,0.0,3158.0,5736.5,22352.5
4,01,01,2019,04:00 - 05:00,302.0,443.50,154.25,0.0,0.0,0.00,...,0.0,0.0,NaN,324.5,6818.0,64.0,0.0,2985.0,5357.0,21751.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,31,12,2019,19:00 - 20:00,220.0,1797.50,162.50,0.0,0.0,1473.75,...,0.0,1051.0,NaN,931.0,6957.0,194.5,0.0,1823.5,3011.5,36604.0
8757,31,12,2019,20:00 - 21:00,220.0,1805.25,162.00,0.0,0.0,676.00,...,0.0,0.0,NaN,863.5,6956.5,155.0,0.0,1843.5,3122.5,34348.0
8758,31,12,2019,21:00 - 22:00,220.0,1778.00,163.25,0.0,0.0,341.00,...,0.0,0.0,NaN,824.0,6960.0,127.0,0.0,1960.5,3269.0,31251.5
8759,31,12,2019,22:00 - 23:00,218.0,1710.50,153.75,0.0,0.0,183.00,...,0.0,0.0,NaN,701.0,6962.0,124.5,0.0,1793.0,3606.0,28597.0
